In [10]:
import time
import random
from tqdm.notebook import tqdm


class Machine:
    
    def __init__(self, vulnerable, percentValue, penetrate, protectionLevel, possibilityProtection):
        self.vulnerable = vulnerable
        self.percentValue = percentValue
        self.penetrate = penetrate
        self.protectionLevel = protectionLevel
        self.possibilityProtection = possibilityProtection
        
    def changeVulnerable(self, vulnerable):
        self.vulnerable = vulnerable
    
    def changeCost(self, percentValue):
        self.percentValue = float(percentValue)
        
    def changePenetrate(self, penetrate):
        self.penetrate = penetrate
    
    def changeProtectionLevel(self, protectionLevel):
        self.protectionLevel = float(protectionLevel)
    
    def changePossibilityProtection(self, possibilityProtection):
        self.possibilityProtection = possibilityProtection
        
    def pr(self):
        print(self.vulnerable, self.percentValue, self.penetrate)

        
def printAllvalues(machineValues, vulnerableMachineValues, importantMachineValues):
    print('machine values = ', machineValues, 'vulnerable machine values = ', vulnerableMachineValues, 'important machine values =', importantMachineValues)
#получаем кол-во машин
def GetMachineValues():
    machineValues = random.randint(4, 15)
    return machineValues
#получаем кол-во уязвимых машин 
def GetVulnerableMachineValues(machineValues):
    vulnerableMachineValues = random.randint(1, machineValues)
    return vulnerableMachineValues
#получаем номер главной машины
def GetImportantMachineValues(machineValues):
    importantMachineValues = random.randint(1, machineValues)
    return importantMachineValues
#получаем процент, который занимает главная машина 
def GetPercentValue():
    percentValue = random.randint(30, 60)
    return percentValue

def GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues):
    machineArray = []
    percentValue = GetPercentValue() / 100
    reversePercent = ((100 - percentValue) / (machineValues - 1)) / 100
    reversePercent = ("%.2f" % reversePercent)
    for i in range(machineValues):
        if i == (importantMachineValues - 1):
            tmp = Machine(False, float(percentValue), False, float(0.0), False)
        else:
            tmp = Machine(False, float(reversePercent), False, float(0.0), False)
        machineArray.append(tmp)
    #здесь баг - может 2 и более раз сгенерироваться
    #одинаковое число и поэтому кол-во машин будет неточным
    for i in range(0, vulnerableMachineValues):
        j = random.randint(0, machineValues - 1)
        machineArray[j].changeVulnerable(True)
    return machineArray
#возвращает число уязвимых машин, так как из-за бага
#прошлое число уязвимых != новому
def GetVulnerableMachine(machineValues, machineArray):
    vulnerableMachine = 0
    for i in range(machineValues):
        if machineArray[i].vulnerable == True:
            vulnerableMachine += 1
    return vulnerableMachine
#это все возможные разрывы функции стоимости на данном промежутке 1-10
def CheckCost(a):
    BreakFunc = {
        '1':'1',
        '2':'0.5',
        '3':'0.333',
        '4':'0.25',
        '5':'0.2',
        '6':'0.166',
        '7':'0.142',
        '8':'0.125',
        '9':'0.111',
        '10':'0.1'
    }
    return BreakFunc[a]
#в этой функции есть разрыв, то что справа от него по x
#то не получится защитить, так как не хватит ресурсов
#этот разрыв в той точке, где знаменатель обращается в 0
def CostFunction(s, a, Ei):
    try:
        return -(s * Ei * Ei)/(a * Ei - 1)
    except ZeroDivisionError:
        return 0

def UtilityDefender(s, a, Ei, machineArray):
    # l - процент понесенных потерь машин
    tmp = 0
    percent = len(machineArray) / 100
    l = 0
    for i in range(len(machineArray)):
        if machineArray[i].penetrate == True:
            l += percent
            tmp += CostFunction(s, a, machineArray[i].protectionLevel)
    return - l - tmp

def UtilityAttacker(machineArray):
    percent = len(machineArray) / 100
    l = 0
    for i in range(len(machineArray)):
        if machineArray[i].penetrate == True:
            l += percent
    return l
    
def AttackerStrategyProportional(machineArray):
    attackerProportionalEffort = []
    for i in range(len(machineArray)):
        tmp = machineArray[i].percentValue
        attackerProportionalEffort.append(tmp)
    return attackerProportionalEffort
#Если в пером раунде атакующий взломал >= 1 машины
#То он перераспределяет свои ресурсы
#отложи эту функцию, пока что протестируем без перераспределения
def AttackerStrategyProportionalRedistribution(machineArray):
    attackerProportionalEffort = []
    for i in range(len(machineArray)):
        if machineArray[i].penetrate == False:
            tmp = machineArray[i].percentValue
            attackerProportionalEffort.append(tmp)
        else:
            attackerProportionalEffort.append(0)
    return attackerProportionalEffort
        
def AttackerStrategyHigh():
    attackerHighEffort = 1
    return attackerHighEffort
        
def DefenderStrategyNothing():
    return None
    
def DefenderStrategyProportional(machineArray, Ei, a):
    a = str(a)
    for i in range(len(machineArray)):
        if float(CheckCost(a)) <= machineArray[i].percentValue * Ei:
            machineArray[i].changePossibilityProtection(False)
        else:
            machineArray[i].changePossibilityProtection(True)
            machineArray[i].changeProtectionLevel("%.2f" % (machineArray[i].percentValue * Ei))
    
def DefenderStrategyHigh(Ei, importantMachineValues, machineArray, a):
    if float(CheckCost(str(a))) <= Ei:
        machineArray[importantMachineValues - 1].changePossibilityProtection(False)
    else:
        machineArray[importantMachineValues - 1].changePossibilityProtection(True)
        machineArray[importantMachineValues - 1].changeProtectionLevel(Ei)
#P-атакующий H-защитник
def ComparisonStrategyPH(machineArray, importantMachineValues, Ei):
    attackerProportionalEffort = AttackerStrategyProportional(machineArray)
    DefenderStrategyHigh(Ei, importantMachineValues, machineArray, a)
    count = 0
    for i in range(len(machineArray)):
        if machineArray[i].vulnerable == True:
            if attackerProportionalEffort[i] > machineArray[i].percentValue or attackerProportionalEffort[importantMachineValues - 1] > machineArray[importantMachineValues - 1].protectionLevel:
                machineArray[i].changePenetrate(True)
                count += 1
    if count > 1:
        return True
    else:
        return False
#P-атакующий P-защитник
def ComparisonStrategyPP(machineArray, Ei, a):
    attackerProportionalEffort = AttackerStrategyProportional(machineArray)
    DefenderStrategyProportional(machineArray, Ei, a)
    count = 0
    for i in range(len(machineArray)):
        if attackerProportionalEffort[i] > machineArray[i].protectionLevel:
            if machineArray[i].vulnerable == True:
                machineArray[i].changePenetrate(True)
                count += 1
    if count > 0:
        return True
    else:
        return False
#P - атакующий N - защитник
def ComparisonStrategyPN(machineArray):
    attackerProportionalEffort = AttackerStrategyProportional(machineArray)
    count = 0
    for i in range(len(machineArray)):
        if attackerProportionalEffort[i] > machineArray[i].percentValue:
            if machineArray[i].vulnerable == True:
                machineArray[i].changePenetrate(True)
                count += 1
    if count > 0:
        return True
    else:
        return False
#H-атакующий P-защитник
def ComparisonStrategyHP(machineArray, Ei, importantMachineValues, a):
    attackerHighEffort = AttackerStrategyHigh()
    DefenderStrategyProportional(machineArray, Ei, a)
    if attackerHighEffort > machineArray[importantMachineValues - 1].protectionLevel:
        if machineArray[importantMachineValues - 1].vulnerable == True:
            machineArray[importantMachineValues - 1].changePenetrate(True)
            return True
        else:
            return False
#H-атакующий H-защитник
def ComparsionStrategyHH(Ei, importantMachineValues, machineArray, a):
    attackerHighEffort = AttackerStrategyHigh()
    DefenderStrategyHigh(Ei, importantMachineValues, machineArray, a)
    if attackerHighEffort > machineArray[importantMachineValues - 1].protectionLevel:
        if machineArray[importantMachineValues - 1].vulnerable == True:
            machineArray[importantMachineValues - 1].changePenetrate(True)
            return True
        else:
            return False
#H-Атакующий N-защитник
def ComparsionStrategyHN(machineArray, importantMachineValues):
    attackerHighEffort = AttackerStrategyHigh()
    if attackerHighEffort > machineArray[importantMachineValues - 1].percentValue:
        if machineArray[importantMachineValues - 1].vulnerable == True:
            machineArray[importantMachineValues - 1].changePenetrate(True)
            return True
        else:
            return False

for s in (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9):
    for a in range(1, 11, 1):
        for strategyA in (1, 2):
            for strategyD in (1, 2, 3):
                for Ei in (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9):
                    if strategyA == 1 and strategyD == 1:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(50000)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparisonStrategyPH(machineArray, importantMachineValues, Ei)
                            if tmp == True:
                                utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                utilityAttacker += UtilityAttacker(machineArray)
                            else:
                                utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                        print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                        print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
                    elif strategyA == 1 and strategyD == 2:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(50000)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparisonStrategyPP(machineArray, Ei, a)
                            if tmp == True:
                                utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                utilityAttacker += UtilityAttacker(machineArray)
                            else:
                                utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                        print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                        print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
                    elif strategyA == 1 and strategyD == 3:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(50000)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparisonStrategyPN(machineArray)
                            if tmp == True:
                                utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                utilityAttacker += UtilityAttacker(machineArray)
                            else:
                                utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                        print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                        print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
                    elif strategyA == 2 and strategyD == 1:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(50000)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparisonStrategyHP(machineArray, Ei, importantMachineValues, a)
                            if tmp == True:
                                utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                utilityAttacker += UtilityAttacker(machineArray)
                            else:
                                utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                        print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                        print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
                    elif strategyA == 2 and strategyD == 2:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(50000)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparsionStrategyHH(Ei, importantMachineValues, machineArray, a)
                            if tmp == True:
                                utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                utilityAttacker += UtilityAttacker(machineArray)
                            else:
                                utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                        print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                        print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
                    elif strategyA == 2 and strategyD == 3:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(50000)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparsionStrategyHN(machineArray, importantMachineValues)
                            if tmp == True:
                                utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                utilityAttacker += UtilityAttacker(machineArray)
                            else:
                                utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                        print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                        print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)

  0%|          | 0/50000 [00:00<?, ?it/s]

for s = 0.1 a = 1 sA =  1 sd = 1 Ei = 0.1
Utility A = 20434.81000000122 utility D = -21039.203333333735


  0%|          | 0/50000 [00:00<?, ?it/s]

for s = 0.1 a = 1 sA =  1 sd = 1 Ei = 0.2
Utility A = 20374.700000001354 utility D = -21054.7650000012


  0%|          | 0/50000 [00:00<?, ?it/s]

...

for s = 0.1 a = 2 sA =  2 sd = 1 Ei = 0.5
Utility A = 1984.0799999996764 utility D = -2205.143559069549


  0%|          | 0/50000 [00:00<?, ?it/s]

for s = 0.1 a = 2 sA =  2 sd = 1 Ei = 0.6
Utility A = 1990.5199999996762 utility D = -2398.803323198943


  0%|          | 0/50000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
class Machine:
    
    def __init__(self, vulnerable, percentValue, penetrate, protectionLevel, possibilityProtection):
        self.vulnerable = vulnerable
        self.percentValue = percentValue
        self.penetrate = penetrate
        self.protectionLevel = protectionLevel
        self.possibilityProtection = possibilityProtection
        
    def changeVulnerable(self, vulnerable):
        self.vulnerable = vulnerable
    
    def changeCost(self, percentValue):
        self.percentValue = percentValue
        
    def changePenetrate(self, penetrate):
        self.penetrate = penetrate
    
    def changeProtectionLevel(self, protectionLevel):
        self.protectionLevel = float(protectionLevel)
    
    def changePossibilityProtection(self, possibilityProtection):
        self.possibilityProtection = possibilityProtection
    
    def pr(self):
        print(type(self.protectionLevel))
        

def DefenderStrategyProportional(machineArray, Ei):
    for i in range(len(machineArray)):
        machineArray[i].changeProtectionLevel("%.2f" % (float(machineArray[i].percentValue) * Ei))

machineArray = []
tmp = Machine(False, 0.1, False, 0, False)
machineArray.append(tmp)
Ei = 12354
DefenderStrategyProportional(machineArray, Ei)
machineArray[0].pr()


<class 'float'>


ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time
You should consider upgrading via the '/home/bkozeev/notebook/notebook/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
